# **Instalação (ubuntu)**

**Fazer as instalações do ODBC 18 e pyodbc para conexão com banco de dados:** 

- No terminal bash: instalar o driver ODBC colocando o código presente no "UBUNTU" em https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline 

In [ ]:
if ! [[ "18.04 20.04 22.04" == *"$(lsb_release -rs)"* ]];
then
    echo "Ubuntu $(lsb_release -rs) is not currently supported.";
    exit;
fi

sudo su
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list

exit
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

In [ ]:
!pip install pyodbc

# Conexão com o banco de dados e importar dados

Considerando que é um script de continuação e que já foi iniciado a sessão do Spark e foi gerado os dataframes

In [12]:
import pyodbc

#Conectar com SQL server
try:
    conn = pyodbc.connect("Driver={ODBC Driver 18 for SQL Server};Server=tcp:<SERVER>,1433;Database=<DATABASE>;Uid=<USER>;Pwd=<PASSWORD>;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;")
    cursor = conn.cursor()
    print("Conexão estabelecida com sucesso!")
except pyodbc.Error as ex:
    print("Erro ao estabelecer conexão:", ex)

#Criar tabela no banco de dados
    # Tabela de clientes
cursor.execute('''
                CREATE TABLE clients (
                    id INT PRIMARY KEY,
                    nome VARCHAR(150),
                    email VARCHAR(150),
                    data_cadastro DATETIME,
                    telefone VARCHAR(150)
                )
                ''')


   # Tabela de transactions
cursor.execute('''
                CREATE TABLE transactions (
                    id INT PRIMARY KEY,
                    cliente_id INT,
                    valor FLOAT,
                    data DATETIME,
                    tipo_transacao VARCHAR(45),
                )
                ''')


# Inserir dados na tabela clients
try:
    for row in df_clientes_order.collect():             # Inserindo uma linha na tabela
        cursor.execute("INSERT INTO clients (id, nome, email, data_cadastro, telefone) VALUES (?, ?, ?, ?, ? )", row[0], row[1], row[2], row[3], row[4])
    print("Dados inseridos na tabela de clientes com sucesso!")

except Exception as e:
    print("Não foi possível inserir dados na tabela clients:", e)

# Inserir dados na tabela transactions
try:
    for row in df_transactions.collect():               # Inserindo uma linha na tabela
        cursor.execute("INSERT INTO transactions (id, cliente_id, valor, data, tipo_transacao) VALUES (?, ?, ?, ?, ?)", row[0], row[1], row[2], row[3], row[4])
    print("Dados inseridos na tabela de transactions com sucesso!")

except Exception as e:
    print("Não foi possível inserir dados na tabela transactions:", e)   
    
   
# Salvando as mudanças
conn.commit()

# Fechando a conexão
conn.close()

# Encerrando a sessão Spark
spark.stop()

Conexão estabelecida com sucesso!
Dados inseridos na tabela de clientes com sucesso!
Dados inseridos na tabela de transactions com sucesso!
